In [1]:
import cv2
import numpy as np
import os
import csv

In [ ]:
def detect_steps(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    equalized = cv2.equalizeHist(blurred)
    edges = cv2.Canny(equalized, 50, 150)
    lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold=120, minLineLength=100, maxLineGap=5)
    
    horizontal_lines = []
    for line in lines:
        x1, y1, x2, y2 = line[0]
        angle = np.arctan2(y2 - y1, x2 - x1) * 180/np.pi
        if abs(angle) < 10:
            horizontal_lines.append(line)
    
    def group_lines(lines, threshold=20):
        grouped = []
        for line in lines:
            _, y1, _, y2 = line[0]
            y_center = (y1 + y2) / 2
            if not grouped:
                grouped.append([y_center, line])
            else:
                added = False
                for group in grouped:
                    if abs(y_center - group[0]) < threshold:
                        group.append(line)
                        added = True
                        break
                if not added:
                    grouped.append([y_center, line])
        return grouped
    
    grouped_lines = group_lines(horizontal_lines)
    return len(grouped_lines)

In [3]:
image_folder = 'images'
csv_file = 'data annotations - Feuille 1.csv'
output_csv = 'resultats.csv'

results = []
correct_predictions = 0
total_predictions = 0

with open(csv_file, mode='r', encoding='utf-8') as file:
    reader = csv.reader(file)

    # on ignore les 2 premieres ligens
    next(reader)
    next(reader)
    
    for row in reader:
        if len(row) < 2 or not row[0]:  # ignorer les lignes vides ou incompletes
            continue
        
        image_name = row[0]
        expected = int(row[1])

        image_path = os.path.join(image_folder, image_name)
        if not os.path.exists(image_path):
            #print(f"Erreur : fichier introuvable {image_name}, pas bien :(")
            continue
        
        # detection des marches
        image = cv2.imread(image_path)
        pred = detect_steps(image)

        is_correct = pred == expected
        if is_correct:
            correct_predictions += 1
        total_predictions += 1
        results.append([image_name, expected, pred, is_correct])
        #print(f"Image : {image_name} | attendues : {expected} | detectees : {pred} | Correct : {is_correct}")

# calcule accuracy
if total_predictions > 0:
    accuracy = correct_predictions / total_predictions
    print(f"Accuracy : {accuracy * 100:.2f}%")
else:
    print("C nul")

# sauvegarde
with open(output_csv, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Image', 'Marches attendues', 'Marches detectees', 'Correct'])
    writer.writerows(results)
print(f"Resultats dans {output_csv}")


Accuracy : 5.95%
Resultats dans resultats.csv
